The first stage to running an encryption is to get the serial and base setup. This configures the serial, the output pins for the trigger, and the buttons and switches.

In [1]:
import sys
!{sys.executable} -m pip install pyserial

In [1]:
print("hello world")

hello world


In [2]:
import time
import serial
from time import sleep

from pynq.overlays.base import BaseOverlay
from pynq.lib.arduino import arduino_io

base = BaseOverlay('base.bit')

# trigger for the pynq on arduino pins a0 and a1
trigger_out = arduino_io.Arduino_IO(base.iop_arduino.mb_info, 14, 'out')
trigger_in = arduino_io.Arduino_IO(base.iop_arduino.mb_info, 15, 'in')

key = b'aesEncryptionKey'  # can change this


In [3]:
# pull in the customlibrary
%run pyaes.ipynb

In [4]:
led0 = base.leds[0] #Corresponds to LED LD0
led1 = base.leds[1] #Corresponds to LED LD1
led2 = base.leds[2] #Corresponds to LED LD2
led3 = base.leds[3] #Corresponds to LED LD3

In [5]:
sw0 = base.switches[0] #Corresponds to SW0
sw1 = base.switches[1] #Corresponds to SW1

In [6]:
btn0 = base.buttons[0] # button 0 hopefully
btn1 = base.buttons[1]
btn2 = base.buttons[2]
btn3 = base.buttons[3]

In [7]:
class CWSerial:
    ACK = b"z00" # 'z00'
    READ = b"r" #'r'
    def __init__(self, device):
        self.device = device
        self.connection = None

    def configure(self):
        self.connection = serial.Serial(
            self.device, 38400, timeout=None, bytesize=serial.EIGHTBITS, parity=serial.PARITY_NONE,
            stopbits=serial.STOPBITS_ONE, xonxoff=False)
        time.sleep(1)

    def read(self, msg_size=1):
        data = self.connection.read(msg_size)
        return data

    def readline(self):
        data = self.connection.readline().strip()
        return data

    # pass write a byte array with cmd and data and newline
    def write(self, cmd):
        print(f"writing {cmd}")
        self.connection.write(cmd)
        



In [8]:
print("Welcome to the pynq z2 board test")
mode = 0 # mode 0 is regular input, mode 1 is file input

Welcome to the pynq z2 board test


The setup is now complete and we can begin encryptions.

In [9]:
def encrypt(key, plaintext):
    encrypter = Encrypter(AESModeOfOperationECB(key))
    ciphertext = encrypter.feed(plaintext)
    ciphertext += encrypter.feed()

    #            print( repr(ciphertext))
#     for letter in ciphertext:
#         print('{:0>2}'.format(str(hex(letter))[2:]), end='')
    return ciphertext

In [10]:
# assumes type bytes, bytes for arguments
def make_cmd(cmd, byte_data=b''):
    buffer = cmd + byte_data + b'\n'
    return buffer

def format_bytestr_as_hexstr(text):
    hexform = ''
    for letter in text:
        hexform += ('{:0>2}'.format(str(hex(letter))[2:]))
    return bytes(hexform, 'utf-8')

def format_hexstr_as_bytestr(hexstr):
    return bytes.fromhex(hexstr.decode('utf-8'))

In [ ]:
# switch based LED toggling for test
data= ''
key = b'aesEncryptionKey'
cw_com = CWSerial("/dev/ttyUSB0")
cw_com.configure()

print("Configured Pynq; can start sending data")
while(True):  # All the code below while(True) runs forever
    # display help
    if btn0.read():
        print("SW1 controls whether lights and encryption or cw encryption")
        print("In encryption mode:")
        print("\t BTN0 resets the data")
        print("\t BTN1 orders input of data\n")
        print("\t BTN2 allows key input (padded to 16 characters)")
        print("\t BTN3 does encryption")

    if not sw1.read():
        led2.toggle()
        led3.toggle()
        if btn1.read():
            led1.toggle()
            led0.toggle()
            trigger_out.write(1)
            data = input()
            x = encrypt(key, str.encode(data))
            trigger_out.write(0)
            print(x)
            
    # run the input read and encryption to the cw
    else:
        #b = mystring.encode('utf-8')
        
        inp = cw_com.readline()
        print(inp)
        
        if inp.decode("utf-8")[0] == 'k':
            key = format_hexstr_as_bytestr(inp[1:])
            cw_com.write(make_cmd(CWSerial.ACK))
            print('received key')
            inp = cw_com.readline()
            
        if inp.decode("utf-8")[0] == 'p':
            data = format_hexstr_as_bytestr(inp[1:])
#            cw_com.write(cw_com.make_cmd(cw_com.ACK))
            print("received plaintext")

        if key and data:
            print(f"ready to encrypt  {data} with key {key}")
            trigger_out.write(1)
            ciphertext = encrypt(key, data)
            ciphertext=ciphertext[:16]

            cw_com.write(make_cmd(CWSerial.READ, format_bytestr_as_hexstr(ciphertext)))
            cw_com.write(make_cmd(CWSerial.ACK))

            trigger_out.write(0)
            print(f"encrypted:  {ciphertext}\n")


Configured Pynq; can start sending data
b'k2a3a45293212982353bc21d2c36d00ff'
writing b'z00\n'
received key
received plaintext
ready to encrypt  b'q>\xe3\xdd\n\xd0Vu\xf9Kv\x00\xd4\x1d\x01\x89' with key b'*:E)2\x12\x98#S\xbc!\xd2\xc3m\x00\xff'
writing b'r7c55a56683baeb48baaa2c06dcca7bfd\n'
writing b'z00\n'
encrypted:  b'|U\xa5f\x83\xba\xebH\xba\xaa,\x06\xdc\xca{\xfd'

b'k2b7e151628aed2a6abf7158809cf4f3c'
writing b'z00\n'
received key
received plaintext
ready to encrypt  b'\x8e.i\x9f!X\xbc\x13\x89\x14x\xecw\xd4C\xb6' with key b'+~\x15\x16(\xae\xd2\xa6\xab\xf7\x15\x88\t\xcfO<'
writing b'r654d4ececabd37599a0f164b86a8d5fb\n'
writing b'z00\n'
encrypted:  b'eMN\xce\xca\xbd7Y\x9a\x0f\x16K\x86\xa8\xd5\xfb'

b'pf74b363e32054494f126341c335db7f5'
received plaintext
ready to encrypt  b'\xf7K6>2\x05D\x94\xf1&4\x1c3]\xb7\xf5' with key b'+~\x15\x16(\xae\xd2\xa6\xab\xf7\x15\x88\t\xcfO<'
writing b'r8a25eac775a9e5b7a3f7d0c8c34751e3\n'
writing b'z00\n'
encrypted:  b'\x8a%\xea\xc7u\xa9\xe5\xb7\xa3\xf7\xd0\x

b'p6e73274b273efaf1984374bdde650f7f'
received plaintext
ready to encrypt  b"ns'K'>\xfa\xf1\x98Ct\xbd\xdee\x0f\x7f" with key b'+~\x15\x16(\xae\xd2\xa6\xab\xf7\x15\x88\t\xcfO<'
writing b'r230d75df6a927e6636b39f788aeae273\n'
writing b'z00\n'
encrypted:  b'#\ru\xdfj\x92~f6\xb3\x9fx\x8a\xea\xe2s'

b'p27feec648c6a0dc59fd9422fae72c4cb'
received plaintext
ready to encrypt  b"'\xfe\xecd\x8cj\r\xc5\x9f\xd9B/\xaer\xc4\xcb" with key b'+~\x15\x16(\xae\xd2\xa6\xab\xf7\x15\x88\t\xcfO<'
writing b'r1f4af3ca43a64be6356bc15ca0799658\n'
writing b'z00\n'
encrypted:  b'\x1fJ\xf3\xcaC\xa6K\xe65k\xc1\\\xa0y\x96X'

b'p812408baa3281950f21b63862c977a35'
received plaintext
ready to encrypt  b'\x81$\x08\xba\xa3(\x19P\xf2\x1bc\x86,\x97z5' with key b'+~\x15\x16(\xae\xd2\xa6\xab\xf7\x15\x88\t\xcfO<'
writing b'r9ef38893133fdc917826f06bfc4d26f9\n'
writing b'z00\n'
encrypted:  b'\x9e\xf3\x88\x93\x13?\xdc\x91x&\xf0k\xfcM&\xf9'

b'pe8246d264dffcd86dae3bacb3ad4ee1e'
received plaintext
ready to encrypt  b'\xe8$m&M\xff\xcd\x

b'pc6253378f133ec1330add166fc5f4039'
received plaintext
ready to encrypt  b'\xc6%3x\xf13\xec\x130\xad\xd1f\xfc_@9' with key b'+~\x15\x16(\xae\xd2\xa6\xab\xf7\x15\x88\t\xcfO<'
writing b'raec8080dbf5b12c715c3be40979c22a8\n'
writing b'z00\n'
encrypted:  b'\xae\xc8\x08\r\xbf[\x12\xc7\x15\xc3\xbe@\x97\x9c"\xa8'

b'pe8682f7bac73e1ba4d65356ed3609969'
received plaintext
ready to encrypt  b'\xe8h/{\xacs\xe1\xbaMe5n\xd3`\x99i' with key b'+~\x15\x16(\xae\xd2\xa6\xab\xf7\x15\x88\t\xcfO<'
writing b'r8c0106cfe4c96db407dc68143fb85835\n'
writing b'z00\n'
encrypted:  b'\x8c\x01\x06\xcf\xe4\xc9m\xb4\x07\xdch\x14?\xb8X5'

b'pe1f57c23fe2570179975a7201312403f'
received plaintext
ready to encrypt  b'\xe1\xf5|#\xfe%p\x17\x99u\xa7 \x13\x12@?' with key b'+~\x15\x16(\xae\xd2\xa6\xab\xf7\x15\x88\t\xcfO<'
writing b'ra3277d2348f14e479f5e6e70a9cd2276\n'
writing b'z00\n'
encrypted:  b'\xa3\'}#H\xf1NG\x9f^np\xa9\xcd"v'

b'p64c19c48801ba170a3d73ca1d3296e27'
received plaintext
ready to encrypt  b"d\xc1\x9cH\x80\x1b\xa1

b'pd0d71deed92b260357bd9ef340c213c8'
received plaintext
ready to encrypt  b'\xd0\xd7\x1d\xee\xd9+&\x03W\xbd\x9e\xf3@\xc2\x13\xc8' with key b'+~\x15\x16(\xae\xd2\xa6\xab\xf7\x15\x88\t\xcfO<'
writing b'r0373fc34e3f4d9092babb6e6e8537a25\n'
writing b'z00\n'
encrypted:  b'\x03s\xfc4\xe3\xf4\xd9\t+\xab\xb6\xe6\xe8Sz%'

b'p83637bc16594ae9ddd2140b001a69490'
received plaintext
ready to encrypt  b'\x83c{\xc1e\x94\xae\x9d\xdd!@\xb0\x01\xa6\x94\x90' with key b'+~\x15\x16(\xae\xd2\xa6\xab\xf7\x15\x88\t\xcfO<'
writing b'r5cc084cec86128973111dd095d0a5dc7\n'
writing b'z00\n'
encrypted:  b'\\\xc0\x84\xce\xc8a(\x971\x11\xdd\t]\n]\xc7'

b'p6f36d1021a928d32657949e03660c3c0'
received plaintext
ready to encrypt  b'o6\xd1\x02\x1a\x92\x8d2eyI\xe06`\xc3\xc0' with key b'+~\x15\x16(\xae\xd2\xa6\xab\xf7\x15\x88\t\xcfO<'
writing b'rf8792817106d0c9a2d4799a9e1767d15\n'
writing b'z00\n'
encrypted:  b'\xf8y(\x17\x10m\x0c\x9a-G\x99\xa9\xe1v}\x15'

b'p9d8076e507da1838228a848b0abc6a93'
received plaintext
ready to encrypt

b'p5d703361e1d5580e0d0bd0b060e3df86'
received plaintext
ready to encrypt  b']p3a\xe1\xd5X\x0e\r\x0b\xd0\xb0`\xe3\xdf\x86' with key b'+~\x15\x16(\xae\xd2\xa6\xab\xf7\x15\x88\t\xcfO<'
writing b'r69f03ab393af8413d0ef46952fb6b930\n'
writing b'z00\n'
encrypted:  b'i\xf0:\xb3\x93\xaf\x84\x13\xd0\xefF\x95/\xb6\xb90'

b'p1f0111ac407baea7c33addb7b43fcd31'
received plaintext
ready to encrypt  b'\x1f\x01\x11\xac@{\xae\xa7\xc3:\xdd\xb7\xb4?\xcd1' with key b'+~\x15\x16(\xae\xd2\xa6\xab\xf7\x15\x88\t\xcfO<'
writing b'rd74cb1fe195d6f8f0f304787da20c740\n'
writing b'z00\n'
encrypted:  b'\xd7L\xb1\xfe\x19]o\x8f\x0f0G\x87\xda \xc7@'

b'p3073df1c7f1327ee51cef6d65d214ecc'
received plaintext
ready to encrypt  b"0s\xdf\x1c\x7f\x13'\xeeQ\xce\xf6\xd6]!N\xcc" with key b'+~\x15\x16(\xae\xd2\xa6\xab\xf7\x15\x88\t\xcfO<'
writing b'rfe1152607e6a799c56f704c2ddb82d06\n'
writing b'z00\n'
encrypted:  b'\xfe\x11R`~jy\x9cV\xf7\x04\xc2\xdd\xb8-\x06'

b'pf1d1bb4390ff2d5cf5acb7b123d6cd75'
received plaintext
ready to encrypt

b'pd9c1d70b14c1689a9d621fd1887bf2cc'
received plaintext
ready to encrypt  b'\xd9\xc1\xd7\x0b\x14\xc1h\x9a\x9db\x1f\xd1\x88{\xf2\xcc' with key b'+~\x15\x16(\xae\xd2\xa6\xab\xf7\x15\x88\t\xcfO<'
writing b'r2eeabb1db2c9244862d3c6082e0c887a\n'
writing b'z00\n'
encrypted:  b'.\xea\xbb\x1d\xb2\xc9$Hb\xd3\xc6\x08.\x0c\x88z'

b'pd921f842070a53d5d82fda2308adda03'
received plaintext
ready to encrypt  b'\xd9!\xf8B\x07\nS\xd5\xd8/\xda#\x08\xad\xda\x03' with key b'+~\x15\x16(\xae\xd2\xa6\xab\xf7\x15\x88\t\xcfO<'
writing b'r69b69c7dfbf4dab1f60c7f56a939a36e\n'
writing b'z00\n'
encrypted:  b'i\xb6\x9c}\xfb\xf4\xda\xb1\xf6\x0c\x7fV\xa99\xa3n'

b'p427a26d5a9cd98e46e416b0dbcbe8fb4'
received plaintext
ready to encrypt  b'Bz&\xd5\xa9\xcd\x98\xe4nAk\r\xbc\xbe\x8f\xb4' with key b'+~\x15\x16(\xae\xd2\xa6\xab\xf7\x15\x88\t\xcfO<'
writing b'r46e0fd8d4c8e5d7299df417ad16375f1\n'
writing b'z00\n'
encrypted:  b'F\xe0\xfd\x8dL\x8e]r\x99\xdfAz\xd1cu\xf1'

b'pd185ed62c494f135de415b71f4302d54'
received plaintext
ready 

b'p49b27e0cedf0aefbba8403b7faeadef7'
received plaintext
ready to encrypt  b'I\xb2~\x0c\xed\xf0\xae\xfb\xba\x84\x03\xb7\xfa\xea\xde\xf7' with key b'+~\x15\x16(\xae\xd2\xa6\xab\xf7\x15\x88\t\xcfO<'
writing b'racba93fc9a9a8c2d1e9305985c83e446\n'
writing b'z00\n'
encrypted:  b'\xac\xba\x93\xfc\x9a\x9a\x8c-\x1e\x93\x05\x98\\\x83\xe4F'

b'pea3f8421d8401f86ab6393c3a3573e45'
received plaintext
ready to encrypt  b'\xea?\x84!\xd8@\x1f\x86\xabc\x93\xc3\xa3W>E' with key b'+~\x15\x16(\xae\xd2\xa6\xab\xf7\x15\x88\t\xcfO<'
writing b'r4d11860579c18ef43de1f26159aff693\n'
writing b'z00\n'
encrypted:  b'M\x11\x86\x05y\xc1\x8e\xf4=\xe1\xf2aY\xaf\xf6\x93'

b'pe81ab94b8fa0d34a738041ea1344d58c'
received plaintext
ready to encrypt  b'\xe8\x1a\xb9K\x8f\xa0\xd3Js\x80A\xea\x13D\xd5\x8c' with key b'+~\x15\x16(\xae\xd2\xa6\xab\xf7\x15\x88\t\xcfO<'
writing b'rf7916360f914b71b65262a128b48fcee\n'
writing b'z00\n'
encrypted:  b'\xf7\x91c`\xf9\x14\xb7\x1be&*\x12\x8bH\xfc\xee'

b'pf0bb6b7646c931a9e8e88f6649e0e7ee'
recei

b'p2786a86efef567787d93dda1b8871fca'
received plaintext
ready to encrypt  b"'\x86\xa8n\xfe\xf5gx}\x93\xdd\xa1\xb8\x87\x1f\xca" with key b'+~\x15\x16(\xae\xd2\xa6\xab\xf7\x15\x88\t\xcfO<'
writing b'r071fefa8ac83a7531c873cdbc38c5e28\n'
writing b'z00\n'
encrypted:  b'\x07\x1f\xef\xa8\xac\x83\xa7S\x1c\x87<\xdb\xc3\x8c^('

b'pd7853abf9ac55285820f3ada68ffeb22'
received plaintext
ready to encrypt  b'\xd7\x85:\xbf\x9a\xc5R\x85\x82\x0f:\xdah\xff\xeb"' with key b'+~\x15\x16(\xae\xd2\xa6\xab\xf7\x15\x88\t\xcfO<'
writing b'rdbb252d5f423e8192ce35f94abd54f9c\n'
writing b'z00\n'
encrypted:  b'\xdb\xb2R\xd5\xf4#\xe8\x19,\xe3_\x94\xab\xd5O\x9c'

b'pb23517d74ea3bba61f09b34024bc6763'
received plaintext
ready to encrypt  b'\xb25\x17\xd7N\xa3\xbb\xa6\x1f\t\xb3@$\xbcgc' with key b'+~\x15\x16(\xae\xd2\xa6\xab\xf7\x15\x88\t\xcfO<'
writing b'r9b6dfcd04d5428d016bb0f43230666f0\n'
writing b'z00\n'
encrypted:  b'\x9bm\xfc\xd0MT(\xd0\x16\xbb\x0fC#\x06f\xf0'

b'pcd06427257bbc0331586e260187494a8'
received plaintext
r

writing b'r66014491fd8f2ce82c737de1073fbbe8\n'
writing b'z00\n'
encrypted:  b'f\x01D\x91\xfd\x8f,\xe8,s}\xe1\x07?\xbb\xe8'

b'p709b867fe43f1055d69330b110bf61bb'
received plaintext
ready to encrypt  b'p\x9b\x86\x7f\xe4?\x10U\xd6\x930\xb1\x10\xbfa\xbb' with key b'+~\x15\x16(\xae\xd2\xa6\xab\xf7\x15\x88\t\xcfO<'
writing b'rf2c50e70bff34a9ce6bca093708ffb3b\n'
writing b'z00\n'
encrypted:  b'\xf2\xc5\x0ep\xbf\xf3J\x9c\xe6\xbc\xa0\x93p\x8f\xfb;'



In [35]:
# try reducing clock


b'\x9234'
b'923334'
